## Inspiration Index

- Data Source:
  - `public_art`
  - `museums`
  - `libraries`
  - `parks_and_facilities`
  - `community_nonprofit_orgs`
  - `faith-based_facilities`

In [2]:
import pandas as pd

# public art
public_art = pd.read_csv('../../data_cleaned/assets/public_art.csv')
public_art_count = public_art['zip_code'].value_counts()
print("public_art:")
print("  mean:", public_art_count.mean())
print("  median:", public_art_count.median())
print("  max:", public_art_count.max())

# museums
museums = pd.read_csv('../../data_cleaned/assets/museums.csv')
museums_count = museums['zip_code'].value_counts()
print("museums:")
print("  mean:", museums_count.mean())
print("  median:", museums_count.median())
print("  max:", museums_count.max())

# libraries
libraries = pd.read_csv('../../data_cleaned/assets/libraries.csv')
libraries_count = libraries['zip_code'].value_counts()
print("libraries:")
print("  mean:", libraries_count.mean())
print("  median:", libraries_count.median())
print("  max:", libraries_count.max())

# parks
parks = pd.read_csv('../../data_cleaned/assets/parks_and_facilities.csv')
parks_count = parks['zip_code'].value_counts()
print("parks:")
print("  mean:", parks_count.mean())
print("  median:", parks_count.median())
print("  max:", parks_count.max())

# nonprofit orgnizations
nonprofit_orgs = pd.read_csv('../../data_cleaned/assets/community_nonprofit_orgs.csv')
nonprofit_orgs_count = nonprofit_orgs['zip_code'].value_counts()
print("nonprofit_orgs:")
print("  mean:", nonprofit_orgs_count.mean())
print("  median:", nonprofit_orgs_count.median())
print("  max:", nonprofit_orgs_count.max())

# faith-based facilities
faith_based = pd.read_csv('../../data_cleaned/assets/faith-based_facilities.csv')
faith_based_count = faith_based['zip_code'].value_counts()
print("faith_based:")
print("  mean:", faith_based_count.mean())
print("  median:", faith_based_count.median())
print("  max:", faith_based_count.max())

public_art:
  mean: nan
  median: nan
  max: nan
museums:
  mean: nan
  median: nan
  max: nan
libraries:
  mean: 1.1363636363636365
  median: 1.0
  max: 2
parks:
  mean: nan
  median: nan
  max: nan
nonprofit_orgs:
  mean: 50.09340659340659
  median: 6.0
  max: 1717
faith_based:
  mean: 9.76595744680851
  median: 8.0
  max: 36


#### Tastescape Score Calculation

- According to the number, we find out a good scale of the combination of the amount of restaurants, coffee shops, and supermarkets in the community, that is 30:3:2
- The score is calculated as follows:
  - `Tastescape Score = (restaurants + coffee_shops * 10 + supermarkets * 5) / 30`
  - We need to normalize the score to 0-100, so we need to find the max and min of the score in the dataset, and calculate the final score

In [3]:
# create a new dataframe with the counts of each zip code
zip_code_counts = pd.DataFrame({
    'zip_code': restaurant_count.index,
    'restaurant_count': restaurant_count.values,
    'coffee_shop_count': coffee_shop_count.reindex(restaurant_count.index, fill_value=0).values,
    'supermarket_count': supermarket_count.reindex(restaurant_count.index, fill_value=0).values
})
# print head 10 of the new dataframe
print(zip_code_counts.head(10))

NameError: name 'restaurant_count' is not defined

In [ ]:
# calculate the score for each zip code
zip_code_counts['score'] = (zip_code_counts['restaurant_count'] +
                             zip_code_counts['coffee_shop_count'] * 10 +
                             zip_code_counts['supermarket_count'] * 15) / 30

# normalize the score to be between 0 and 1
zip_code_counts['score'] = (zip_code_counts['score'] - zip_code_counts['score'].min()) / (zip_code_counts['score'].max() - zip_code_counts['score'].min())
# sort the dataframe by score in descending order
zip_code_counts = zip_code_counts.sort_values(by='score', ascending=False)
# print head 10 of the new dataframe
print(zip_code_counts.head(10))

# save the new dataframe to a csv file
tastescape_scores = zip_code_counts[['zip_code', 'score']]
print(tastescape_scores.head(10))
tastescape_scores.to_csv('../../data_score/tastescape_scores.csv', index=False)

    zip_code  restaurant_count  coffee_shop_count  supermarket_count     score
0      15222               246                 18                  4  1.000000
4      15237               139                  9                  8  0.717526
1      15212               213                  6                  3  0.653608
2      15213               157                 10                  1  0.558763
6      15146               127                  4                  7  0.558763
7      15205               125                  5                  6  0.544330
10     15206                83                 10                  4  0.498969
8      15108               109                  6                  4  0.470103
12     15217                76                  9                  4  0.463918
3      15219               153                  7                  0  0.457732
    zip_code     score
0      15222  1.000000
4      15237  0.717526
1      15212  0.653608
2      15213  0.558763
6      15146  0.